# Load Data

run save data cells in projet.ipynb before

In [1]:
import pandas as pd

movies = pd.read_csv('../data/save/movies.csv')
ratings = pd.read_csv('../data/save/ratings.csv')
users = pd.read_csv('../data/save/users.csv')

In [2]:
display(movies.head())
display(ratings.head())
display(users.head())

,MovieId,Title,Year,isAdult,runtimeMinutes,averageRating,numVotes,Action,Adult,Adventure,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,1,toy story,1995,0,81,4.15,1073870,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2,jumanji,1995,0,104,3.55,379231,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3,grumpier old men,1995,0,101,3.30,29839,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,4,waiting to exhale,1995,0,124,3.00,12277,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,5,father of the bride part ii,1995,0,106,3.05,41877,0,0,0,...,0,0,0,0,1,0,0,0,0,0


,UserId,MovieId,Rating
0,1,1193,5.0
1,1,661,3.0
2,1,914,3.0
3,1,3408,4.0
4,1,2355,5.0


,UserId,Gender,Age
0,1,1,1
1,2,0,56
2,3,0,25
3,4,0,45
4,5,0,25


# NNE

Raison : Résultat pas satisfaisant

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, Flatten, Concatenate, Input
from tensorflow.keras.optimizers import Adam

In [ ]:
full_data = pd.merge(pd.merge(ratings, users, on='UserId'), movies, on='MovieId')

# Sélectionner les colonnes pour item_train et user_train
item_features = movies.columns.difference(['Title', 'Year', 'runtimeMinutes', 'MovieId']).tolist()
item_train = full_data[item_features].values
user_train = full_data[['Gender', 'Age']].values
y_train = full_data['Rating'].values

# Définir les dimensions des entrées utilisateur et item
num_user_features = user_train.shape[1]
num_item_features = item_train.shape[1]

# Sauvegarder les données non scalées
item_train_unscaled = item_train.copy()
user_train_unscaled = user_train.copy()
y_train_unscaled = y_train.copy()

# Appliquer le scaling
scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))

# Inverser les transformations pour vérification
inverse_item = scalerItem.inverse_transform(item_train)
inverse_user = scalerUser.inverse_transform(user_train)

# Vérifier que les scalers peuvent inverser correctement les transformations
assert np.allclose(item_train_unscaled, inverse_item), "Something is wrong with the item scaler"
assert np.allclose(user_train_unscaled, inverse_user), "Something is wrong with the user scaler"

print("\033[92m" + "All tests passed!" + "\033[0m")

# Split dataset
user_item_data = np.concatenate([user_train, item_train], axis=1)
X_train, X_test, y_train, y_test = train_test_split(
    user_item_data, y_train, train_size=0.80, shuffle=True, random_state=1
)

user_train = X_train[:, :num_user_features]
item_train = X_train[:, num_user_features:]
user_test = X_test[:, :num_user_features]
item_test = X_test[:, num_user_features:]

print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

# Définir le modèle
num_outputs = 32
tf.random.set_seed(1)

user_NN = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(num_outputs),
    ]
)

item_NN = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(num_outputs),
    ]
)

# Create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features,))
vu = user_NN(input_user)
vu = tf.keras.layers.Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))(vu)

# Create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features,))
vm = item_NN(input_item)
vm = tf.keras.layers.Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))(vm)

# Compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# Specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

# Compiler le modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraîner le modèle
history = model.fit([user_train, item_train], y_train, epochs=2, batch_size=32, validation_split=0.2)

# Évaluer le modèle
loss = model.evaluate([user_test, item_test], y_test)
print(f'Test Loss: {loss}')


In [ ]:
def predict_for_user(user_id, model, users, movies, scalerUser, scalerItem, scalerTarget):
    # Extraire les informations de l'utilisateur
    user_data = users[users['UserId'] == user_id].iloc[:, 1:].values
    # Mettre à l'échelle les données utilisateur
    user_data = scalerUser.transform(user_data)

    # Extraire les informations des films
    movie_data = movies.drop(columns=['Title', 'Year', 'runtimeMinutes', 'MovieId']).values
    # Mettre à l'échelle les données des films
    movie_data = scalerItem.transform(movie_data)

    # Répliquer les données utilisateur pour chaque film
    user_data = np.repeat(user_data, movie_data.shape[0], axis=0)

    # Faire les prédictions
    predictions = model.predict([user_data, movie_data])

    # Inverser la transformation des prédictions
    predictions = scalerTarget.inverse_transform(predictions)

    # Créer un DataFrame avec les résultats
    results = pd.DataFrame({
        'MovieId': movies['MovieId'],
        'Title': movies['Title'],
        'PredictedRating': predictions.flatten()
    })

    return results

user_id = 98
predictions = predict_for_user(user_id, model, users, movies, scalerUser, scalerItem, scalerTarget)
print(predictions)

In [ ]:
print('average predicted rating:', predictions['PredictedRating'].mean())
print('max predicted rating:', predictions['PredictedRating'].max())
print('min predicted rating:', predictions['PredictedRating'].min())

# Hybrid ALS

In [3]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20, 13)
%matplotlib inline
%config InlineBackend.figure_format = "retina"

In [4]:
# Create a Spark session
spark = SparkSession.builder.appName("ALSMatrixFactorisation").getOrCreate()

In [5]:
ratings_spark = spark.createDataFrame(ratings)
users_spark = spark.createDataFrame(users)
movies_spark = spark.createDataFrame(movies)

# Merge all dataframes
df_pred = ratings_spark.join(users_spark, on='UserId').join(movies_spark, on='MovieId')

In [6]:
indexer = [
    StringIndexer(inputCol=column, outputCol=column + "_index") 
    for column in ['UserId', 'MovieId']
]

In [7]:
all_genres = ['Action', 'Adult', 'Adventure', 'Animation', 'Biography', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western']

assembler = VectorAssembler(
    inputCols=['averageRating'] + all_genres, # Include averageRating and genre columns
    outputCol="features"
)

In [8]:
pipeline = Pipeline(stages=indexer + [assembler])
transformed = pipeline.fit(df_pred).transform(df_pred)
transformed.show()

+-------+------+------+------+---+--------------------+----+-------+--------------+-------------+--------+------+-----+---------+---------+---------+----------+------+-----+-----------+-----+------+-------+---------+-------+------+-----+-------+-------+-------+------+-----+--------+---+-------+------------+-------------+--------------------+
|MovieId|UserId|Rating|Gender|Age|               Title|Year|isAdult|runtimeMinutes|averageRating|numVotes|Action|Adult|Adventure|Animation|Biography|Children's|Comedy|Crime|Documentary|Drama|Family|Fantasy|Film-Noir|History|Horror|Music|Musical|Mystery|Romance|Sci-Fi|Sport|Thriller|War|Western|UserId_index|MovieId_index|            features|
+-------+------+------+------+---+--------------------+----+-------+--------------+-------------+--------+------+-----+---------+---------+---------+----------+------+-----+-----------+-----+------+-------+---------+-------+------+-----+-------+-------+-------+------+-----+--------+---+-------+------------+----

In [9]:
(training, test) = transformed.randomSplit([0.8, 0.2], seed=42)